In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K

In [9]:
img_width, img_height = 150, 150
base_dir = './teeth_images'

train_data_dir = base_dir

epochs = 20
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [10]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

c:\Users\julia\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 148, 148, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 72, 72, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 34, 34, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,183,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,212,513 (4.63 MB)

 Trainable params: 1,212,513 (4.63 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
from tensorflow.keras.metrics import Recall, Precision

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', Recall(name='recall'), Precision(name='precision')])

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples


Found 415 images belonging to 2 classes.
Found 102 images belonging to 2 classes.


In [14]:
history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.5023 - loss: 0.9336 - precision: 0.5254 - recall: 0.6388

c:\Users\julia\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 435ms/step - accuracy: 0.4937 - loss: 0.7857 - precision: 0.5358 - recall: 0.7644 - val_accuracy: 0.5833 - val_loss: 0.6907 - val_precision: 0.5833 - val_recall: 1.0000
Epoch 2/20
 1/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.6875 - loss: 0.6875 - precision: 0.6875 - recall: 1.0000

c:\Users\julia\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6875 - loss: 0.6875 - precision: 0.6875 - recall: 1.0000 - val_accuracy: 0.5625 - val_loss: 0.6909 - val_precision: 0.5625 - val_recall: 1.0000
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step - accuracy: 0.5564 - loss: 0.6902 - precision: 0.5606 - recall: 0.9867 - val_accuracy: 0.5729 - val_loss: 0.6913 - val_precision: 0.5729 - val_recall: 1.0000
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6875 - loss: 0.6900 - precision: 0.6875 - recall: 1.0000 - val_accuracy: 0.5625 - val_loss: 0.6914 - val_precision: 0.5625 - val_recall: 1.0000
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - accuracy: 0.5564 - loss: 0.6940 - precision: 0.5640 - recall: 0.9558 - val_accuracy: 0.5833 - val_loss: 0.6883 - val_precision: 0.5833 - val_recall: 1.0000
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6250 - loss: 0.6829 - precision: 0.6250 - recall: 1.0000 - val_accuracy: 0.5521 - val_loss: 0.6892 - val_pre

In [15]:
model.save_weights('dental_pathology_binary_v1.weights.h5')

In [18]:
test_data_dir = './test_images'

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

scores = model.evaluate(test_generator)

print(f"Loss: {scores[0]:.4f}")
print(f"Accuracy: {scores[1]*100:.2f}%")
print(f"Recall: {scores[2]*100:.2f}%")
print(f"Precision: {scores[3]*100:.2f}%")

test_recall = scores[2]
test_precision = scores[3]

if (test_precision + test_recall) > 0:
    f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
    print(f"F1-score: {f1_score:.4f}")
else:
    print("Precision + Recall = 0")

Found 517 images belonging to 2 classes.


33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.5687 - loss: 0.6849 - precision: 0.5687 - recall: 1.0000
Loss: 0.6849
Accuracy: 56.87%
Recall: 100.00%
Precision: 56.87%
F1-score: 0.7250
